## Load Earth System Science Data (ESSD) datasets

Copyright (C) 2021 OS-Climate

Permission is hereby granted, free of charge, to any person obtaining a copy of this software and associated documentation files (the "Software"), to deal in the Software without restriction, including without limitation the rights to use, copy, modify, merge, publish, distribute, sublicense, and/or sell copies of the Software, and to permit persons to whom the Software is furnished to do so, subject to the following conditions:

The above copyright notice and this permission notice shall be included in all copies or substantial portions of the Software.

THE SOFTWARE IS PROVIDED "AS IS", WITHOUT WARRANTY OF ANY KIND, EXPRESS OR IMPLIED, INCLUDING BUT NOT LIMITED TO THE WARRANTIES OF MERCHANTABILITY, FITNESS FOR A PARTICULAR PURPOSE AND NONINFRINGEMENT. IN NO EVENT SHALL THE AUTHORS OR COPYRIGHT HOLDERS BE LIABLE FOR ANY CLAIM, DAMAGES OR OTHER LIABILITY, WHETHER IN AN ACTION OF CONTRACT, TORT OR OTHERWISE, ARISING FROM, OUT OF OR IN CONNECTION WITH THE SOFTWARE OR THE USE OR OTHER DEALINGS IN THE SOFTWARE.

Contributed by Michael Tiemann (Github: MichaelTiemannOSC)

Load Credentials

In [ ]:
# From the AWS Account page, copy the export scripts from the appropriate role using the "Command Line or Programmatic Access" link
# Paste the copied text into ~/credentials.env

from dotenv import dotenv_values, load_dotenv
import os
import pathlib
import sys

dotenv_dir = os.environ.get('CREDENTIAL_DOTENV_DIR', os.environ.get('PWD', '/opt/app-root/src'))
dotenv_path = pathlib.Path(dotenv_dir) / 'credentials.env'
if os.path.exists(dotenv_path):
    load_dotenv(dotenv_path=dotenv_path,override=True)

# import sys
# sys.path.append('../src/')

In [ ]:
# import re
import pandas as pd
import numpy as np

import openpyxl

import pint
import pint_pandas
import iam_units
from openscm_units import unit_registry
pint_pandas.PintType.ureg = unit_registry
ureg = unit_registry

In [ ]:
from osc_ingest_trino import *
import pyarrow as pa
import pyarrow.parquet as pq
import json
import io
import uuid

Create an S3 resource for the bucket holding source data

In [ ]:
import boto3
s3_resource = boto3.resource(
    service_name="s3",
    endpoint_url=os.environ['S3_LANDING_ENDPOINT'],
    aws_access_key_id=os.environ['S3_LANDING_ACCESS_KEY'],
    aws_secret_access_key=os.environ['S3_LANDING_SECRET_KEY'],
)
bucket = s3_resource.Bucket(os.environ['S3_LANDING_BUCKET'])

In [ ]:
# Create an S3 client.  We will user later when we write out data and metadata
s3 = boto3.client(
    service_name="s3",
    endpoint_url=os.environ['S3_DEV_ENDPOINT'],
    aws_access_key_id=os.environ['S3_DEV_ACCESS_KEY'],
    aws_secret_access_key=os.environ['S3_DEV_SECRET_KEY'],
)

In [ ]:
import trino

conn = trino.dbapi.connect(
    host=os.environ['TRINO_HOST'],
    port=int(os.environ['TRINO_PORT']),
    user=os.environ['TRINO_USER'],
    http_scheme='https',
    auth=trino.auth.JWTAuthentication(os.environ['TRINO_PASSWD']),
    verify=True,
)
cur = conn.cursor()

# Show available schemas to ensure trino connection is set correctly
cur.execute('show schemas in osc_datacommons_dev')
cur.fetchall()

In [ ]:
ingest_uuid = str(uuid.uuid4())

custom_meta_key_fields = 'metafields'
custom_meta_key = 'metaset'

schemaname = 'essd'
cur.execute('create schema if not exists osc_datacommons_dev.' + schemaname)
cur.fetchall()

For osc_datacommons_dev, a trino pipeline is a parquet data stored in the S3_DEV_BUCKET
It is a 5-step process to get there from a pandas dataframe

In [ ]:
def create_trino_pipeline (s3, schemaname, tablename, timestamp, df, meta_fields, meta_content):
    global ingest_uuid
    global custom_meta_key_fields, custom_meta_key
    
    # First convert dataframe to pyarrow for type conversion and basic metadata
    table = pa.Table.from_pandas(df)
    # Second, since pyarrow tables are immutable, create a new table with additional combined metadata
    if meta_fields or meta_content:
        meta_json_fields = json.dumps(meta_fields)
        meta_json = json.dumps(meta_content)
        existing_meta = table.schema.metadata
        combined_meta = {
            custom_meta_key_fields.encode(): meta_json_fields.encode(),
            custom_meta_key.encode(): meta_json.encode(),
            **existing_meta
        }
        table = table.replace_schema_metadata(combined_meta)
    # Third, convert table to parquet format (which cannot be written directly to s3)
    pq.write_table(table, '/tmp/{sname}.{tname}.{uuid}.{timestamp}.parquet'.format(sname=schemaname, tname=tablename, uuid=ingest_uuid, timestamp=timestamp))
    # df.to_parquet('/tmp/{sname}.{tname}.{uuid}.parquet'.format(sname=schemaname, tname=tablename, uuid=ingest_uuid, index=False))
    # Fourth, put the parquet-ified data into our S3 bucket for trino.  We cannot compute parquet format directly to S3 but we can copy it once computed
    s3.upload_file(
        Bucket=os.environ['S3_DEV_BUCKET'],
        Key='trino/{sname}/{tname}/{uuid}/{timestamp}/{tname}.parquet'.format(sname=schemaname, tname=tablename, uuid=ingest_uuid, timestamp=timestamp),
        Filename='/tmp/{sname}.{tname}.{uuid}.{timestamp}.parquet'.format(sname=schemaname, tname=tablename, uuid=ingest_uuid, timestamp=timestamp)
    )
    # Finally, create the trino table backed by our parquet files enhanced by our metadata
    cur.execute('.'.join(['drop table if exists osc_datacommons_dev', schemaname, tablename]))
    print('dropping table: ' + tablename)
    cur.fetchall()
    
    schema = create_table_schema_pairs(df)

    tabledef = """create table if not exists osc_datacommons_dev.{sname}.{tname}(
{schema}
) with (
    format = 'parquet',
    external_location = 's3a://{bucket}/trino/{sname}/{tname}/{uuid}/{timestamp}'
)""".format(schema=schema,bucket=os.environ['S3_DEV_BUCKET'],sname=schemaname,tname=tablename,uuid=ingest_uuid,timestamp=timestamp)
    print(tabledef)

    # tables created externally may not show up immediately in cloud-beaver
    cur.execute(tabledef)
    cur.fetchall()

Load ESSD data file using pandas *read_excel* and using *ingest_uuid* as the global UUID for this ingestion

# Do we really want to add UUID to every row, or better to put into table-level metadata?

In [ ]:
import io

bObj = bucket.Object('ESSD/essd_ghg_data.xlsx')
essd_bytes = io.BytesIO(bucket.Object(bObj.key).get()['Body'].read())
timestamp = bObj.last_modified.isoformat()

# It takes almost a minute (!) to load nearly 600K rows of data
%time essd_df = pd.read_excel(essd_bytes, sheet_name='data', converters={'year': lambda x: pd.to_datetime(x, format='%Y')}, dtype={'gwp100_ar5':'int32', 'value':'float64'}, engine='openpyxl')

display(essd_df.columns)

Construct the combined metadata by merging existing table metadata and custom metadata.
Note: The metadata content must be JSON serialisable and encoded as bytes; the metadata key must also be encoded as bytes.

In [ ]:
essd_content_df = pd.read_excel(essd_bytes, sheet_name='info', header=None).dropna(axis=0).set_index(0).T
custom_meta_content = {
    'title': 'Earth System Science Data (ESSD) Dataset',
    'author': essd_content_df[['Author & contact']].squeeze(),
    'contact': essd_content_df[['Author & contact']].squeeze(),
    'description': essd_content_df[['Data description']].squeeze(),
    'release_date': essd_content_df[['Last date of compilation']].squeeze(),
    # How should we describe our transformative step here?
}

essd_metadata_df = pd.read_excel(essd_bytes, sheet_name='metadata')
custom_meta_fields = { d['Variable']: {k:v for k,v in d.items() if k!='Variable' and v==v} for d in essd_metadata_df.to_dict('records') }

The schemaname for this table is `essd`.  Dunno if it's a good idea or simply redundant to prefix the tablename with `essd_`

In [ ]:
tablename = 'ghg_data'
create_trino_pipeline (s3, schemaname, tablename, timestamp, essd_df, custom_meta_fields, custom_meta_content)

Verify that we can restore data and metadata

In [ ]:
# Read the Parquet file into an Arrow table
obj = s3.get_object(
    Bucket=os.environ['S3_DEV_BUCKET'], 
    Key='trino/{sname}/{tname}/{uuid}/{timestamp}/{tname}.parquet'.format(sname=schemaname, tname=tablename, uuid=ingest_uuid, timestamp=timestamp)
)
restored_table = pq.read_table(io.BytesIO(obj['Body'].read()))
# Call the table’s to_pandas conversion method to restore the dataframe
# This operation uses the Pandas metadata to reconstruct the dataFrame under the hood
restored_df = restored_table.to_pandas()
# The custom metadata is accessible via the Arrow table’s metadata object
# Use the custom metadata key used earlier (taking care to once again encode the key as bytes)
restored_meta_json = restored_table.schema.metadata[custom_meta_key.encode()]
# Deserialize the json string to get back metadata
restored_meta = json.loads(restored_meta_json)
# Use the custom metadata fields key used earlier (taking care to once again encode the key as bytes)
restored_meta_json_fields = restored_table.schema.metadata[custom_meta_key_fields.encode()]
# Deserialize the json string to get back metadata
restored_meta_fields = json.loads(restored_meta_json_fields)

In [ ]:
restored_df

In [ ]:
restored_meta

In [ ]:
restored_meta_fields

Grab Sector, Region, 100yr GWPs, and GH4_gwps from one of the two main ESSD data tables (they are the same in both).

In [ ]:
essd_sectors_df = pd.read_excel(essd_bytes, sheet_name='sector_classification', engine='openpyxl')

display(essd_sectors_df.columns)

In [ ]:
custom_meta_content = {
    'title': 'Earth System Science Data (ESSD) Sector Classification',
    'author': essd_content_df[['Author & contact']].squeeze(),
    'contact': essd_content_df[['Author & contact']].squeeze(),
    'description': essd_content_df[['Sectors']].squeeze(),
    'release_date': essd_content_df[['Last date of compilation']].squeeze(),
    # How should we describe our transformative step here?
}

custom_meta_fields = None

tablename = 'sectors'
create_trino_pipeline (s3, schemaname, tablename, timestamp, essd_df, custom_meta_fields, custom_meta_content)

In [ ]:
essd_regions_df = pd.read_excel(essd_bytes, sheet_name='region_classification', engine='openpyxl')

display(essd_regions_df.columns)

In [ ]:
custom_meta_content = {
    'title': 'Earth System Science Data (ESSD) Region Classification',
    'author': essd_content_df[['Author & contact']].squeeze(),
    'contact': essd_content_df[['Author & contact']].squeeze(),
    'description': essd_content_df[['Regions']].squeeze(),
    'release_date': essd_content_df[['Last date of compilation']].squeeze(),
    # How should we describe our transformative step here?
}

custom_meta_fields = None

tablename = 'regions'
create_trino_pipeline (s3, schemaname, tablename, timestamp, essd_df, custom_meta_fields, custom_meta_content)

Now deal with the gas species.  We'll annotate our dataframe with PINT units

In [ ]:
essd_gwp_df = pd.read_excel(essd_bytes, sheet_name='100_yr_gwps', dtype={'gwp_ar5':'int32'}, engine='openpyxl')

display(essd_gwp_df.columns)

In [ ]:
ureg.define("CH4_Combustion = nan CH4")
ureg.define("CH4_Fugitive = nan CH4")
ureg.define("CH4_Process = nan CH4")

ch4_dict = {
    'CH4 Biogenic': 'CH4',
    'CH4 Fossil (Combustion)': 'CH4_Combustion',
    'CH4 Fossil (Fugitive)': 'CH4_Fugitive',
    'CH4 Fossil (Process)': 'CH4_Process'
}

def convert_gas_to_pint_species(s):
    """For a series S of gas species, return the species name known to pint"""
    return [ch4_dict[g] if g in ch4_dict else g.replace('c-', 'C').replace('-','') for g in s.tolist()]

essd_gwp_df = essd_gwp_df.assign(units=lambda x: convert_gas_to_pint_species(x.gas))

In [ ]:
custom_meta_content = {
    'title': 'Earth System Science Data (ESSD) Gas Species',
    'author': essd_content_df[['Author & contact']].squeeze(),
    'contact': essd_content_df[['Author & contact']].squeeze(),
    'description': essd_content_df[['Regions']].squeeze(),
    'release_date': essd_content_df[['Last date of compilation']].squeeze(),
    # How should we describe our transformative step here?
}

custom_meta_fields = None

tablename = 'gwp_100yr'
create_trino_pipeline (s3, schemaname, tablename, timestamp, essd_gwp_df, custom_meta_fields, custom_meta_content)

In [ ]:
essd_ch4_df = pd.read_excel(essd_bytes, sheet_name='CH4_gwps', dtype={'gwp_ar5':'int32', 'subsector':'str'}, engine='openpyxl')

display(essd_ch4_df.columns)

In [ ]:
custom_meta_content = {
    'title': 'Earth System Science Data (ESSD) CH4 GWPs',
    'author': essd_content_df[['Author & contact']].squeeze(),
    'contact': essd_content_df[['Author & contact']].squeeze(),
    'description': essd_content_df[['Regions']].squeeze(),
    'release_date': essd_content_df[['Last date of compilation']].squeeze(),
    # How should we describe our transformative step here?
}

custom_meta_fields = None

tablename = 'ch4_gwp'
create_trino_pipeline (s3, schemaname, tablename, timestamp, essd_gwp_df, custom_meta_fields, custom_meta_content)

In [ ]:
custom_meta_content = {
    'title': 'Earth System Science Data (ESSD) Gas Species',
    'author': essd_content_df[['Author & contact']].squeeze(),
    'contact': essd_content_df[['Author & contact']].squeeze(),
    'description': essd_content_df[['Regions']].squeeze(),
    'release_date': essd_content_df[['Last date of compilation']].squeeze(),
    # How should we describe our transformative step here?
}

custom_meta_fields = None

tablename = 'gwp_100yr'
create_trino_pipeline (s3, schemaname, tablename, timestamp, essd_gwp_df, custom_meta_fields, custom_meta_content)

In [ ]:
import io

bObj = bucket.Object('ESSD/essd_ghg_data_gwp100.xlsx')
essd_gwp100_bytes = io.BytesIO(bucket.Object(bObj.key).get()['Body'].read())
timestamp = bObj.last_modified.isoformat()

%time essd_gwp100_df = pd.read_excel(essd_gwp100_bytes, sheet_name='data', converters={'year': lambda x: pd.to_datetime(x, format='%Y')}, dtype={'gwp100_ar5':'int32', 'value':'float64'}, engine='openpyxl')

display(essd_gwp100_df.columns)

In [ ]:
essd_gwp100_content_df = pd.read_excel(essd_gwp100_bytes, sheet_name='info', header=None).dropna(axis=0).set_index(0).T
custom_meta_content = {
    'title': 'Earth System Science Data (ESSD) GWP100 Dataset',
    'author': essd_content_df[['Author & contact']].squeeze(),
    'contact': essd_content_df[['Author & contact']].squeeze(),
    'description': essd_content_df[['Data description']].squeeze(),
    'release_date': essd_content_df[['Last date of compilation']].squeeze(),
    # How should we describe our transformative step here?
}

essd_gwp100_metadata_df = pd.read_excel(essd_gwp100_bytes, sheet_name='metadata')
custom_meta_fields = { d['Variable']: {k:v for k,v in d.items() if k!='Variable' and v==v} for d in essd_metadata_df.to_dict('records') }

In [ ]:
custom_meta_content

Construct the combined metadata by merging existing table metadata and custom metadata.
Note: The metadata content must be JSON serialisable and encoded as bytes; the metadata key must also be encoded as bytes.

In [ ]:
tablename = 'gwp100_data'
create_trino_pipeline (s3, schemaname, tablename, timestamp, essd_gwp100_df, custom_meta_fields, custom_meta_content)

In [ ]:
# Read the Parquet file into an Arrow table
obj = s3.get_object(
    Bucket=os.environ['S3_DEV_BUCKET'], 
    Key='trino/{sname}/{tname}/{uuid}/{timestamp}/{tname}.parquet'.format(sname=schemaname, tname=tablename, uuid=ingest_uuid, timestamp=timestamp)
)
restored_table = pq.read_table(io.BytesIO(obj['Body'].read()))
# Call the table’s to_pandas conversion method to restore the dataframe
# This operation uses the Pandas metadata to reconstruct the dataFrame under the hood
restored_df = restored_table.to_pandas()
# The custom metadata is accessible via the Arrow table’s metadata object
# Use the custom metadata key used earlier (taking care to once again encode the key as bytes)
restored_meta_json = restored_table.schema.metadata[custom_meta_key.encode()]
# Deserialize the json string to get back metadata
restored_meta = json.loads(restored_meta_json)
# Use the custom metadata fields key used earlier (taking care to once again encode the key as bytes)
restored_meta_json_fields = restored_table.schema.metadata[custom_meta_key_fields.encode()]
# Deserialize the json string to get back metadata
restored_meta_fields = json.loads(restored_meta_json_fields)

In [ ]:
restored_df

In [ ]:
bObj = bucket.Object('ESSD/essd_lulucf_data.xlsx')
lulucf_bytes = io.BytesIO(bucket.Object(bObj.key).get()['Body'].read())
timestamp = bObj.last_modified.isoformat()

lulucf_df = pd.read_excel(lulucf_bytes, sheet_name='data', converters={'year': lambda x: pd.to_datetime(x, format='%Y')}, engine='openpyxl')

display(lulucf_df.columns)

In [ ]:
lulucf_content_df = pd.read_excel(lulucf_bytes, sheet_name='info', header=None).dropna(axis=0).set_index(0).T
custom_meta_content = {
    'title': 'Earth System Science Data (ESSD) Land Use Change and Forestry Dataset',
    'author': lulucf_content_df[['Author']].squeeze(),
    'contact': lulucf_content_df[['Contact']].squeeze(),
    'description': lulucf_content_df[['Info']].squeeze(),
    'release_date': essd_content_df[['Last date of compilation']].squeeze(),
    # How should we describe our transformative step here?
}

custom_meta_fields = { 'Units': { k:'tCO2' for k in ['blue', 'houghton', 'oscar', 'mean'] }}

In [ ]:
custom_meta_content

Construct the combined metadata by merging existing table metadata and custom metadata.
Note: The metadata content must be JSON serialisable and encoded as bytes; the metadata key must also be encoded as bytes.

In [ ]:
tablename = 'lulucf'
create_trino_pipeline (s3, schemaname, tablename, timestamp, lulucf_df, custom_meta_fields, custom_meta_content)

Restore data and metadata

In [ ]:
# Read the Parquet file into an Arrow table
obj = s3.get_object(
    Bucket=os.environ['S3_DEV_BUCKET'], 
    Key='trino/{sname}/{tname}/{uuid}/{timestamp}/{tname}.parquet'.format(sname=schemaname, tname=tablename, uuid=ingest_uuid, timestamp=timestamp)
)
restored_table = pq.read_table(io.BytesIO(obj['Body'].read()))
# Call the table’s to_pandas conversion method to restore the dataframe
# This operation uses the Pandas metadata to reconstruct the dataFrame under the hood
restored_df = restored_table.to_pandas()
# The custom metadata is accessible via the Arrow table’s metadata object
# Use the custom metadata key used earlier (taking care to once again encode the key as bytes)
restored_meta_json = restored_table.schema.metadata[custom_meta_key.encode()]
# Deserialize the json string to get back metadata
restored_meta = json.loads(restored_meta_json)
# Use the custom metadata fields key used earlier (taking care to once again encode the key as bytes)
restored_meta_json_fields = restored_table.schema.metadata[custom_meta_key_fields.encode()]
# Deserialize the json string to get back metadata
restored_meta_fields = json.loads(restored_meta_json_fields)

In [ ]:
restored_df

In [ ]:
# cur.execute('create schema if not exists osc_datacommons_dev.' + schemaname)

cur.execute('show tables in osc_datacommons_dev.essd')
cur.fetchall()

## Load metadata following an ingestion process into trino metadata store

### The schema is *metastore*, and the table names are *meta_schema*, *meta_table*, *meta_field*

In [ ]:
# Create metastore structure
metastore = {'catalog':'osc_datacommons_dev',
             'schema':'essd',
             'table':tablename,
             'metadata':custom_meta_content,
             'uuid':ingest_uuid}
# Create DataFrame
df_meta = pd.DataFrame(metastore)
# Print the output
df_meta

# ??? Now what?